In [122]:
import json
import os
import time
import re
import logging
import random

from itertools import chain
from string import punctuation


import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import (AdamW, T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup)
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

from termcolor import colored
from sklearn.model_selection import train_test_split
from pathlib import Path

In [2]:
pl.seed_everything(42)

Global seed set to 42


42

In [7]:
with Path("./datasets/QA/BioASQ/BioASQ-train-factoid-4b.json").open() as json_file:
    data = json.load(json_file)

In [10]:
data.keys()

dict_keys(['data', 'version'])

In [ ]:
data["version"]

In [12]:
data["data"][0].keys()

dict_keys(['paragraphs', 'title'])

In [14]:
data["data"][0]["title"]

'BioASQ6b'

In [16]:
len(data["data"][0]["paragraphs"])

3266

In [17]:
questions = data["data"][0]["paragraphs"]

In [18]:
questions[0]

{'qas': [{'id': '52bf208003868f1b06000019_002',
   'question': 'What is the inheritance pattern of Li–Fraumeni syndrome?',
   'answers': [{'text': 'autosomal dominant', 'answer_start': 213}]}],
 'context': 'Balanced t(11;15)(q23;q15) in a TP53+/+ breast cancer patient from a Li-Fraumeni syndrome family. Li-Fraumeni Syndrome (LFS) is characterized by early-onset carcinogenesis involving multiple tumor types and shows autosomal dominant inheritance. Approximately 70% of LFS cases are due to germline mutations in the TP53 gene on chromosome 17p13.1. Mutations have also been found in the CHEK2 gene on chromosome 22q11, and others have been mapped to chromosome 11q23. While characterizing an LFS family with a documented defect in TP53, we found one family member who developed bilateral breast cancer at age 37 yet was homozygous for wild-type TP53. Her mother also developed early-onset primary bilateral breast cancer, and a sister had unilateral breast cancer and a soft tissue sarcoma. Cytog

In [19]:
def extract_questions_and_answer(factoid_path: Path):
    with factoid_path.open() as json_file:
        data = json.load(json_file)

    questions = data["data"][0]["paragraphs"]
    data_rows = []

    for question in questions:
        context = question["context"]
        for qa in question["qas"]:
            question_text = qa["question"]
            answer_text = qa["answers"]
            for answer in answer_text:
                answer_text = answer["text"]
                answer_start = answer["answer_start"]
                answer_end = answer_start + len(answer_text)

                data_rows.append({
                    "question": question_text,
                    "context": context,
                    "answer": answer_text,
                    "answer_start": answer_start,
                    "answer_end": answer_end
                })

    return pd.DataFrame(data_rows)

In [21]:
extract_questions_and_answer(Path("./datasets/QA/BioASQ/BioASQ-train-factoid-4b.json")).head()

,question,context,answer,answer_start,answer_end
0,What is the inheritance pattern of Li–Fraumeni...,Balanced t(11;15)(q23;q15) in a TP53+/+ breast...,autosomal dominant,213,231
1,What is the inheritance pattern of Li–Fraumeni...,Genetic modeling of Li-Fraumeni syndrome in ze...,autosomal dominant,105,123
2,Which type of lung cancer is afatinib used for?,Clinical perspective of afatinib in non-small ...,EGFR-mutant NSCLC,1203,1220
3,Which hormone abnormalities are characteristic...,"DOCA sensitive pendrin expression in kidney, h...",thyroid,419,426
4,Which hormone abnormalities are characteristic...,Clinical and molecular characteristics of Pend...,thyroid,705,712


In [25]:
factoid_paths = sorted(list(Path("./datasets/QA/BioASQ").glob("BioASQ-train-factoid-*")))
factoid_paths

[PosixPath('datasets/QA/BioASQ/BioASQ-train-factoid-4b.json'),
 PosixPath('datasets/QA/BioASQ/BioASQ-train-factoid-5b.json'),
 PosixPath('datasets/QA/BioASQ/BioASQ-train-factoid-6b.json'),
 PosixPath('datasets/QA/BioASQ/BioASQ-train-factoid-7b.json')]

In [27]:
dfs = []
for factoid_path in factoid_paths:
    dfs.append(extract_questions_and_answer(factoid_path))

df = pd.concat(dfs)

In [28]:
df.head()

,question,context,answer,answer_start,answer_end
0,What is the inheritance pattern of Li–Fraumeni...,Balanced t(11;15)(q23;q15) in a TP53+/+ breast...,autosomal dominant,213,231
1,What is the inheritance pattern of Li–Fraumeni...,Genetic modeling of Li-Fraumeni syndrome in ze...,autosomal dominant,105,123
2,Which type of lung cancer is afatinib used for?,Clinical perspective of afatinib in non-small ...,EGFR-mutant NSCLC,1203,1220
3,Which hormone abnormalities are characteristic...,"DOCA sensitive pendrin expression in kidney, h...",thyroid,419,426
4,Which hormone abnormalities are characteristic...,Clinical and molecular characteristics of Pend...,thyroid,705,712


In [29]:
df.shape

(17219, 5)

In [30]:
len(df.question.unique())

557

In [31]:
len(df.context.unique())

6161

In [33]:
sample_question = df.iloc[240]
sample_question

In [42]:
def color_answer(question):
    answer_start, answer_end = question["answer_start"], question["answer_end"]
    context = question["context"]

    return colored(context[:answer_start], "white") + colored(context[answer_start:answer_end], "green") + colored(context[answer_end:], "white")

In [41]:
print(color_answer(sample_question))

Adductor laryngeal breathing dystonia in a patient with lubag (X-linked dystonia-Parkinsonism syndrome). We report a patient with Lubag (X-linked dystonia-parkinsonism) who presented with severe respiratory stridor from adductor laryngeal breathing dystonia. Emergency tracheostomy was necessary, and subsequent laryngeal injection with botulinum toxin led to worsening aspiration. Botulinum toxin injection for severe lingual dystonia was successful.


In [43]:
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)


/Users/drw/venv_miniforge310/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [45]:
sample_encoding = tokenizer(
    "Would I rather be feared or loved?",
    "Easy. Both. I want people to be afraid of how much they love me."
)
sample_encoding

{'input_ids': [5328, 27, 1066, 36, 3, 27625, 42, 1858, 58, 1, 6844, 5, 2867, 5, 27, 241, 151, 12, 36, 7403, 13, 149, 231, 79, 333, 140, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [49]:
preds = [
    tokenizer.decode(input_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
    for input_id in sample_encoding["input_ids"]
]

In [50]:
" ".join(preds)

'Would I rather be  feared or loved ? </s> Easy . Both . I want people to be afraid of how much they love me . </s>'

In [51]:
encoding = tokenizer(
    sample_question["question"],
    sample_question["context"],
    max_length=512,
    padding="max_length",
    truncation="only_second",
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)

In [52]:
encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [53]:
tokenizer.decode(encoding["input_ids"].squeeze())

'What is the synonym of the lubag disease?</s> Adductor laryngeal breathing dystonia in a patient with lubag (X-linked dystonia-Parkinsonism syndrome). We report a patient with Lubag (X-linked dystonia-parkinsonism) who presented with severe respiratory stridor from adductor laryngeal breathing dystonia. Emergency tracheostomy was necessary, and subsequent laryngeal injection with botulinum toxin led to worsening aspiration. Botulinum toxin injection for severe lingual dystonia was successful.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [55]:
sample_question

question                What is the synonym of the lubag disease?
context         Adductor laryngeal breathing dystonia in a pat...
answer                             X-linked dystonia-Parkinsonism
answer_start                                                   63
answer_end                                                     93
Name: 240, dtype: object

In [56]:
answer_encoding = tokenizer(
    sample_question["answer"],
    max_length=32,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)

In [57]:
tokenizer.decode(answer_encoding["input_ids"].squeeze())

'X-linked dystonia-Parkinsonism</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [58]:
labels = answer_encoding["input_ids"]

In [61]:
labels

tensor([[    3,     4,    18, 29000, 16633,    17,  8008,    18, 13212,  7815,
            32, 14378,     1,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100]])

In [60]:
labels[labels == 0] = -100

In [ ]:
labels

In [149]:
class BioQADataset(Dataset):
    def __init__(
            self,
            data: pd.DataFrame,
            tokenizer: T5Tokenizer,
            source_max_token_len: int = 396,
            target_max_token_len: int = 32,
            ):

        self.tokenizer = tokenizer
        self.data = data

        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]
        source_encoding = tokenizer(
            data_row["question"],
            data_row["context"],
            max_length=self.source_max_token_len,
            padding="max_length",
            truncation="only_second",
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

        target_encoding = tokenizer(
            data_row["answer"],
            max_length=self.target_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

        labels = target_encoding["input_ids"]
        labels[labels == 0] = -100

        return dict(
            question=data_row["question"],
            context=data_row["context"],
            answer=data_row["answer"],
            input_ids=source_encoding["input_ids"].flatten(),
            attention_mask=source_encoding["attention_mask"].flatten(),
            labels=labels.flatten()
        )

In [150]:
sample_dataset = BioQADataset(df, tokenizer)

In [68]:
for data in sample_dataset:
    print(data["question"])
    print(data["answer"])
    print(data["input_ids"][:10])
    print(data["labels"][:10])
    break

What is the inheritance pattern of Li–Fraumeni syndrome?
autosomal dominant
tensor([  363,    19,     8, 28915,  3275,    13,  1414,   104,   371,  6340])
tensor([ 1510, 10348,   138, 12613,     1,  -100,  -100,  -100,  -100,  -100])


In [69]:
train_df, val_df = train_test_split(df, test_size=0.05)


In [70]:
train_df.shape, val_df.shape


((16358, 5), (861, 5))

In [144]:
class BioQADataModule(pl.LightningDataModule):
    def __init__(
            self,
            train_df: pd.DataFrame,
            val_df: pd.DataFrame,
            tokenizer: T5Tokenizer,
            batch_size: int = 8,
            source_max_token_len: int = 396,
            target_max_token_len: int = 32,
            ):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def setup(self, stage=None):
        self.train_dataset = BioQADataset(
            data=self.train_df,
            tokenizer=self.tokenizer,
            source_max_token_len=self.source_max_token_len,
            target_max_token_len=self.target_max_token_len
        )

        self.val_dataset = BioQADataset(
            self.val_df,
            self.tokenizer,
            self.source_max_token_len,
            self.target_max_token_len
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=4
        )

In [74]:
smaller_df = df.drop_duplicates(subset=["context"], inplace=False)

In [75]:
smaller_df.shape

(6161, 5)

In [77]:
train_df, val_df = train_test_split(smaller_df, test_size=0.05)

In [146]:
BATCH_SIZE = 2
N_EPOCHS = 1

data_module = BioQADataModule(train_df=train_df, val_df=val_df, tokenizer=tokenizer, batch_size=BATCH_SIZE)

In [147]:
data_module.setup()

## Training

In [80]:
model = T5ForConditionalGeneration.from_pretrained(model_name, return_dict=True)

In [105]:
input_ids = tokenizer(
    "translate English to German: What is the name of my dog?",
    return_tensors="pt"
).input_ids

In [106]:
generated_ids = model.generate(input_ids=input_ids)

In [107]:
preds = [
    tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    for g in generated_ids
]

In [108]:
" ".join(preds)

'Was ist der Name meines Hundes?'

In [109]:
text = """
Prompt Engineering, also known as In-Context Prompting, refers to methods for how to communicate with LLM to steer its behavior for desired outcomes without updating the model weights. It is an empirical science and the effect of prompt engineering methods can vary a lot among models, thus requiring heavy experimentation and heuristics.

This post only focuses on prompt engineering for autoregressive language models, so nothing with Cloze tests, image generation or multimodality models. At its core, the goal of prompt engineering is about alignment and model steerability. Check my previous post on controllable text generation.

[My personal spicy take] In my opinion, some prompt engineering papers are not worthy 8 pages long, since those tricks can be explained in one or a few sentences and the rest is all about benchmarking. An easy-to-use and shared benchmark infrastructure should be more beneficial to the community. Iterative prompting or external tool use would not be trivial to set up. Also non-trivial to align the whole research community to adopt it.

Useful Resources

OpenAI Cookbook has many in-depth examples for how to utilize LLM efficiently.
LangChain, a library for combining language models with other components to build applications.
Prompt Engineering Guide repo contains a pretty comprehensive collection of education materials on prompt engineering.
learnprompting.org
PromptPerfect
Semantic Kernel
Basic Prompting
Zero-shot and few-shot learning are two most basic approaches for prompting the model, pioneered by many LLM papers and commonly used for benchmarking LLM performance.

Zero-Shot
Zero-shot learning is to simply feed the task text to the model and ask for results.

(All the sentiment analysis examples are from SST-2)

Text: i'll bet the video game is a lot more fun than the film.
Sentiment:
Few-shot
Few-shot learning presents a set of high-quality demonstrations, each consisting of both input and desired output, on the target task. As the model first sees good examples, it can better understand human intention and criteria for what kinds of answers are wanted. Therefore, few-shot learning often leads to better performance than zero-shot. However, it comes at the cost of more token consumption and may hit the context length limit when input and output text are long.
"""

In [110]:
input_ids = tokenizer(
    "summarize: " + text,
    return_tensors="pt"
).input_ids

In [111]:
generated_ids = model.generate(input_ids=input_ids)

preds = [
    tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    for g in generated_ids
]

In [112]:
"".join(preds)

'prompt engineering is a method for communicating with an autoregressive language model. it'

In [113]:
model.config

T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "pre

In [115]:
output = model(
    input_ids=encoding["input_ids"],
    attention_mask=encoding["attention_mask"],
    labels=labels
)

In [116]:
output.logits.shape

torch.Size([1, 32, 32128])

In [117]:
output.loss

tensor(2.2242, grad_fn=<NllLossBackward0>)

In [119]:
class BioQAModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(model_name, return_dict=True)
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=5e-5)

In [120]:
model = BioQAModel()

/Users/drw/venv_miniforge310/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [126]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

In [130]:
trainer = pl.Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=N_EPOCHS,
    enable_progress_bar=True
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [151]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [155]:
%tensorboard --logdir lightning_logs/

Launching TensorBoard...

In [154]:
trainer.fit(model, data_module)

/Users/drw/venv_miniforge310/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'BioQADataset' on <module '__main__' (built-in)>
/Users/drw/venv_miniforge310/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
